In [ ]:
!pip install kaggle

In [ ]:
%sh
mkdir /root/.kaggle

In [ ]:
%sh
echo "{\"username\":\"saran\", \"key\":\"0b82666877dae3f74d4a58f894a9dc4c\"}" > /root/.kaggle/kaggle.json

In [ ]:
%sh
ls /root/.kaggle
cat /root/.kaggle/kaggle.json

In [ ]:
%sh
mkdir -p /dbfs/tmp

In [ ]:
%sh
chmod 600 /root/.kaggle/kaggle.json

In [ ]:
%sh
cd /dbfs/tmp
kaggle datasets download -d shikhapatel/predictive-maintenance

In [ ]:
%sh
ls /dbfs/tmp

In [ ]:
%sh
cd /dbfs/tmp
unzip predictive-maintenance.zip

In [ ]:
file_dir = "/data"
file_dir_path1 = f"{file_dir}/Final_dataframes/df_r1b_t5.csv"
dbutils.fs.cp("file:/dbfs/tmp/Final_dataframes/df_r1b_t5.csv", file_dir_path1)

In [ ]:
file_dir = "/data"
file_dir_path2 = f"{file_dir}/Final_dataframes/df_r2b_t5.csv"
dbutils.fs.cp("file:/dbfs/tmp/Final_dataframes/df_r2b_t5.csv", file_dir_path2)

In [ ]:
file_dir = "/data"
file_dir_path3 = f"{file_dir}/Final_dataframes/df_r3b_t5.csv"
dbutils.fs.cp("file:/dbfs/tmp/Final_dataframes/df_r3b_t5.csv", file_dir_path3)

In [ ]:
file_dir = "/data"
file_dir_path4 = f"{file_dir}/Final_dataframes/df_r4b_t5.csv"
dbutils.fs.cp("file:/dbfs/tmp/Final_dataframes/df_r4b_t5.csv", file_dir_path4)

In [ ]:
file_dir = "/data"
file_dir_path5 = f"{file_dir}/Final_dataframes/df_rs_t5.csv"
dbutils.fs.cp("file:/dbfs/tmp/Final_dataframes/df_rs_t5.csv", file_dir_path5)

In [ ]:
df_r1b_t5 = spark.read.csv(file_dir_path1, header=True)
df_r2b_t5 = spark.read.csv(file_dir_path2, header=True)
df_r3b_t5 = spark.read.csv(file_dir_path3, header=True)
df_r4b_t5 = spark.read.csv(file_dir_path4, header=True)
df_rs_t5 = spark.read.csv(file_dir_path5, header=True)


In [ ]:
display(df_r1b_t5)

In [ ]:
display(df_r2b_t5)

In [ ]:
display(df_r3b_t5)

In [ ]:
display(df_r4b_t5)

In [ ]:
display(df_rs_t5)

In [ ]:
display(df_rs_t5.summary())
display(df_r1b_t5.summary())
display(df_r2b_t5.summary())
display(df_r3b_t5.summary())
display(df_r4b_t5.summary())

In [ ]:
from pyspark.sql.functions import lit
from functools import reduce
from pyspark.sql import DataFrame
import pyspark.sql.functions 
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import RandomForestClassifier
import seaborn as sns
import pandas as pd
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import MultilayerPerceptronClassifier
import matplotlib.pyplot as plt

Creating new column 'Status' for each dataset depending on their working status

In [ ]:
value = 'good'
df_rs_t5 = df_rs_t5.withColumn("Status", lit(value))

In [ ]:
value = 'One_bar_broken'
df_r1b_t5 = df_r1b_t5.withColumn("Status", lit(value))

In [ ]:
value = 'two_bars_broken'
df_r2b_t5 = df_r2b_t5.withColumn("Status", lit(value))

In [ ]:
value = 'three_bars_broken'
df_r3b_t5 = df_r3b_t5.withColumn("Status", lit(value))

In [ ]:
value = 'four_bars_broken'
df_r4b_t5 = df_r4b_t5.withColumn("Status", lit(value))

In [ ]:
dfs = [df_rs_t5, df_r1b_t5, df_r2b_t5, df_r3b_t5, df_r4b_t5]
df = reduce(DataFrame.unionAll, dfs)

In [ ]:
display(df)

In [ ]:
df.printSchema()

In [ ]:
# Converting the string data types to float for the below columns

df = df.withColumn("Ia", df["Ia"].cast("float"))
df = df.withColumn("Ib", df["Ib"].cast("float"))
df = df.withColumn("Ic", df["Ic"].cast("float"))
df = df.withColumn("Va", df["Va"].cast("float"))
df = df.withColumn("Vb", df["Vb"].cast("float"))
df = df.withColumn("Vc", df["Vc"].cast("float"))
df = df.withColumn("Trig", df["Trig"].cast("float"))
df = df.withColumn("Vib_acpe", df["Vib_acpe"].cast("float"))
df = df.withColumn("Vib_carc", df["Vib_carc"].cast("float"))
df = df.withColumn("Vib_acpi", df["Vib_acpi"].cast("float"))
df = df.withColumn("Vib_axial", df["Vib_axial"].cast("float"))
df = df.withColumn("Vib_base", df["Vib_base"].cast("float"))

In [ ]:
df.printSchema()

In [ ]:
df2 = df.groupBy("Status").count()
display(df2)

Vactor Assembler for numeric columns

In [ ]:
numericCols = [ 'Ia', 'Ib', 'Ic', 'Va', 'Vb', 'Vc', 'Trig', 'Vib_acpe', 'Vib_carc', 'Vib_acpi', 'Vib_axial', 'Vib_base']
assembler = VectorAssembler(inputCols=numericCols, outputCol="features")
df = assembler.transform(df)
display(df)

String Indexer for category Values

In [ ]:
label_stringIdx = StringIndexer(inputCol = 'Status', outputCol = 'labelIndex')
df = label_stringIdx.fit(df).transform(df)
display(df)

Dividing data into training and testing

In [ ]:
train, test = df.randomSplit([0.7, 0.3], seed = 2021)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Random Forest Classification Model

In [ ]:
# Intializing Model 
rf = RandomForestClassifier(featuresCol ='features', labelCol ='labelIndex', numTrees=5)

# Traning
rfModel = rf.fit(train)

# Tesitng
predictions = rfModel.transform(test)

display(predictions)


In [ ]:
# Comparing predictions with actual values

df_predict = predictions.select('labelIndex','prediction')
pdf = df_predict.toPandas()
display(pdf)

In [ ]:
#Preparing Data for plotting values

data1 = predictions.groupby("labelIndex").count()
data1 = data1.toPandas()
data1 = data1.sort_values(by='labelIndex')
data2 = predictions.groupby("prediction").count()
data2 = data2.toPandas()
data2 = data2.sort_values(by='prediction')

Graph of Actual Vs Predicted values derived from Random Forest Classifier

In [ ]:
# Plotting Actual and Prediction results

myexplode = [0.0, 0.02, 0.03, 0.0, 0.0]
colors = ['#B99095','#FCB5AC','#B5E5CF','#3D5B59', '#29A0B1']
labels = ['Good', 'One_bar_broken', 'Two_bar_broken', 'Three_bar_broken', 'Four_bar_broken']
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(10,5))
ax1.pie(x=data1['count'], autopct="%.1f%%", pctdistance=0.5, startangle = 90, explode=myexplode, colors= colors, radius =1.2)
ax1.set_title('Actual')
ax2.pie(x=data2['count'], autopct="%.1f%%", pctdistance=0.5, startangle = 90, explode=myexplode, colors= colors, radius=1.2)
ax2.set_title('Predicted')
fig.legend(labels=labels, loc='lower center', ncol=5)
plt.show() #show all figures


In [ ]:
# Calculating Accuracy for Random Forest Classifier

evaluator = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

Decision Tree Classification Model

In [ ]:
# Model Inialization
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'labelIndex', maxDepth = 10)

# Training
dtModel = dt.fit(train)

# Testing 
dt_predictions = dtModel.transform(test)

display(dt_predictions)

In [ ]:
# Calculating Accuracy for Decision Tree Classification Model

evaluator = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction")
accuracy = evaluator.evaluate(dt_predictions)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))


In [ ]:
df_predict1 = dt_predictions.select('labelIndex','prediction')
pdf1 = df_predict1.toPandas()
display(pdf1)


In [ ]:
#Preparing Data for plotting values

data1 = dt_predictions.groupby("labelIndex").count()
data1 = data1.toPandas()
data1 = data1.sort_values(by='labelIndex')
data2 = dt_predictions.groupby("prediction").count()
data2 = data2.toPandas()
data2 = data2.sort_values(by='prediction')

Graph of Actual Vs Predicted values derived from Decision Tree Classifier

In [ ]:
# Plotting Actual and Prediction results

myexplode = [0.0, 0.02, 0.03, 0.0, 0.0]
colors = ['#B99095','#FCB5AC','#B5E5CF','#3D5B59', '#29A0B1']
labels = ['Good', 'One_bar_broken', 'Two_bar_broken', 'Three_bar_broken', 'Four_bar_broken']
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(10,5))
ax1.pie(x=data1['count'], autopct="%.1f%%", pctdistance=0.5, startangle = 90, explode=myexplode, colors= colors, radius =1.2)
ax1.set_title('Actual')
ax2.pie(x=data2['count'], autopct="%.1f%%", pctdistance=0.5, startangle = 90, explode=myexplode, colors= colors, radius=1.2)
ax2.set_title('Predicted')
fig.legend(labels=labels, loc='lower center', ncol=5)
plt.show() #show all figures

Cross validation to improve Random Forest Results

In [ ]:
# Creating a multiclass evaluator
evaluator1 = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction")

#building a parameter grid wwith 2 grids
paramGrid = (ParamGridBuilder()
            .addGrid(rf.maxDepth, [2, 4, 6])
            .addGrid(rf.numTrees, [10, 100])
            .build())

#Intinalizing cross validator
cv = CrossValidator(estimator=rf, 
                    evaluator=evaluator1, 
                    estimatorParamMaps=paramGrid, 
                    numFolds=3, 
                    parallelism=25, 
                    seed=42)

# Creating Pipeline with stages
pipeline1 = Pipeline(stages=[cv])
pipelineModel = pipeline1.fit(train)
predDF = pipelineModel.transform(test)

In [ ]:
# Evaluating Accuracy

evaluator = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction")
accuracy = evaluator.evaluate(predDF)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

In [ ]:
cv_predictions = predDF.select('labelIndex','prediction')
cv_pdf = cv_predictions.toPandas()
display(cv_pdf)

In [ ]:
#Preparing Data for plotting values

data1 = predDF.groupby("labelIndex").count()
data1 = data1.toPandas()
data1 = data1.sort_values(by='labelIndex')
data2 = predDF.groupby("prediction").count()
data2 = data2.toPandas()
data2 = data2.sort_values(by='prediction')

Graph of Actual Vs Predicted values derived from Cross Validation

In [ ]:
# Plotting Actual and Prediction results

myexplode = [0.0, 0.02, 0.03, 0.0, 0.0]
colors = ['#B99095','#FCB5AC','#B5E5CF','#3D5B59', '#29A0B1']
labels = ['Good', 'One_bar_broken', 'Two_bar_broken', 'Three_bar_broken', 'Four_bar_broken']
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(10,5))
ax1.pie(x=data1['count'], autopct="%.1f%%", pctdistance=0.5, startangle = 90, explode=myexplode, colors= colors, radius =1.2)
ax1.set_title('Actual')
ax2.pie(x=data2['count'], autopct="%.1f%%", pctdistance=0.5, startangle = 90, explode=myexplode, colors= colors, radius=1.2)
ax2.set_title('Predicted')
fig.legend(labels=labels, loc='lower center', ncol=5)
plt.show() #show all figures

After hyperparameter tuning the random forest model accuracy improved from 69% to 71%

Neural Network Model

In [ ]:
# layers for the neural network:
# input layer of size 12 (features), Three intermediate of size 15 each, output layer of size 5 (classes)
layers = [12, 15, 15, 15, 5]

# creating the trainer and setting its parameters
trainer = MultilayerPerceptronClassifier(maxIter=250, layers=layers, blockSize=128, seed=42, featuresCol = 'features', labelCol = 'labelIndex')

# training the model
model = trainer.fit(train)

result = model.transform(test)

predictionAndLabels = result.select("prediction", "labelIndex")

evaluator = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction")


In [ ]:
# computing accuracy on the test set

accuracy = evaluator.evaluate(predictionAndLabels)

print("Accuracy = %s" % (accuracy))

print("Test Error = %s" % (1.0 - accuracy))

In [ ]:
#Preparing Data for plotting values

data1 = predictionAndLabels.groupby("labelIndex").count()
data1 = data1.toPandas()
data1 = data1.sort_values(by='labelIndex')
data2 = predictionAndLabels.groupby("prediction").count()
data2 = data2.toPandas()
data2 = data2.sort_values(by='prediction')

Graph of Actual Vs Predicted values derived from Neural Network Multilayer Perceptron Classifier

In [ ]:
# Plotting Actual and Prediction results

myexplode = [0.0, 0.02, 0.03, 0.0, 0.0]
colors = ['#B99095','#FCB5AC','#B5E5CF','#3D5B59', '#29A0B1']
labels = ['Good', 'One_bar_broken', 'Two_bar_broken', 'Three_bar_broken', 'Four_bar_broken']
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(10,5))
ax1.pie(x=data1['count'], autopct="%.1f%%", pctdistance=0.5, startangle = 90, explode=myexplode, colors= colors, radius =1.2)
ax1.set_title('Actual')
ax2.pie(x=data2['count'], autopct="%.1f%%", pctdistance=0.5, startangle = 90, explode=myexplode, colors= colors, radius=1.2)
ax2.set_title('Predicted')
fig.legend(labels=labels, loc='lower center', ncol=5)
plt.show() #show all figures

5.0 Autoassociative neural network

In [ ]:
# layers for the neural network:
# input layer of size 12 (features), three intermediate of size 15, 10, 15 respectively, output layer of size 5 (classes)
layers_1 = [12, 15, 10, 15, 5]

# creating the trainer and setting its parameters
trainer_1 = MultilayerPerceptronClassifier(maxIter=250, layers=layers_1, blockSize=128, seed=42, featuresCol = 'features', labelCol = 'labelIndex')

# training the model
model_1 = trainer_1.fit(train)

result_1 = model_1.transform(test)

predictionAndLabels_1 = result_1.select("prediction", "labelIndex")

evaluator_1 = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction")


In [ ]:
# Computing accuracy on the test set

accuracy_1 = evaluator_1.evaluate(predictionAndLabels_1)

print("Accuracy = %s" % (accuracy_1))

print("Test Error = %s" % (1.0 - accuracy_1))

In [ ]:
#Preparing Data for plotting values

data1 = predictionAndLabels_1.groupby("labelIndex").count()
data1 = data1.toPandas()
data1 = data1.sort_values(by='labelIndex')
data2 = predictionAndLabels_1.groupby("prediction").count()
data2 = data2.toPandas()
data2 = data2.sort_values(by='prediction')

Graph of Actual Vs Predicted values derived from Auto Associative Neural Network (Multilayer Perceptron Classifier)

In [ ]:
# Plotting Actual and Prediction results

myexplode = [0.0, 0.02, 0.03, 0.0, 0.0]
colors = ['#B99095','#FCB5AC','#B5E5CF','#3D5B59', '#29A0B1']
labels = ['Good', 'One_bar_broken', 'Two_bar_broken', 'Three_bar_broken', 'Four_bar_broken']
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(10,5))
ax1.pie(x=data1['count'], autopct="%.1f%%", pctdistance=0.5, startangle = 90, explode=myexplode, colors= colors, radius =1.2)
ax1.set_title('Actual')
ax2.pie(x=data2['count'], autopct="%.1f%%", pctdistance=0.5, startangle = 90, explode=myexplode, colors= colors, radius=1.2)
ax2.set_title('Predicted')
fig.legend(labels=labels, loc='lower center', ncol=5)
plt.show() #show all figures

Different Models Comparision

In [ ]:
Models = ['RandomForestClassifier', 'DecisionTreeClassifier', 'CrossValidator(RF)', 'MultilayerPerceptronClassifier', 'MultilayerPerceptronClassifier_AutoAssociative']
Accuracy = [67.7, 81.0, 71.8, 77.6, 76.7]
fig = plt.figure(figsize=(15,5))
ax = fig.add_axes([0,0,1,1])
bar_graph = ax.bar(Models, Accuracy, color='royalblue')
for p in bar_graph:
  height = p.get_height()
  ax.text(x=p.get_x() + p.get_width() / 2, y=height+.80,
      s="{}%".format(height),
      ha='center')

font1 = {'family':'serif','color':'royalblue','size':15}
font2 = {'family':'serif','color':'royalblue','size':15}

plt.xlabel("Models", fontdict = font1)
plt.ylabel("Accuracy Percentage", fontdict = font1)
plt.title("Different Models Accuracy Comparision", loc = 'left')
plt.show()